# https://github.com/wenzhixin/bootstrap-table
https://stackoverflow.com/questions/39003732/display-django-pandas-dataframe-in-a-django-template

* Pandas 가공 결과를 시각화 바로 원할하게 출력하기
* 바로 판다스 출력이 예제로 되어있는데
* 내가 원하는 형태로 가공 후 결과물 출력하기를 알아보자

In [1]:
import pandas as pd

In [2]:
print(pd.DataFrame({1:[1,2,3,4,5], 2:[12,3,4,5,12]}).to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>1</th>
      <th>2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>12</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>3</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>4</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>5</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>12</td>
    </tr>
  </tbody>
</table>


In [3]:
%%html
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>1</th>
      <th>2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>12</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>3</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>4</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>5</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>12</td>
    </tr>
  </tbody>
</table>

,1,2
0,1,12
1,2,3
2,3,4
3,4,5
4,5,12


# **lambda 수식의 활용**
큰 데이터를 처리시, for 문에 비해 훨씬 효과가 빠르다 <br/>
https://dojang.io/mod/page/view.php?id=1059 <br/>
http://book.pythontips.com/en/latest/lambdas.html <br/>
https://stackoverflow.com/questions/31206106/compare-each-item-of-two-lists-in-python

In [4]:
# lambda 를 활용한 계산
add = lambda x, y: x + y
add(3, 5)

8

In [5]:
# lambda 를 활용한 정렬
a = [(1, 2), (4, 1), (9, 10), (13, -3)]
a.sort(key=lambda x: x[1])
a

[(13, -3), (4, 1), (1, 2), (9, 10)]

In [6]:
list1, list2 = (1,2,3,4,5), (10,20,30,40,50)
data = zip(list1, list2)
# data.sort()  # .sort는 오류를 발생한다
# data
list1, list2 = map(lambda t: list(t), zip(*data))
print("list1 : {}\nlist2 : {}".format(list1, list2))

list1 : [1, 2, 3, 4, 5]
list2 : [10, 20, 30, 40, 50]


In [7]:
# itertools 을 활용
import itertools
list_1 = [1,5,4]
list_2 = [2,3,4]
# using list comprehensions
comparisons = [a == b for (a, b) in itertools.product(list_1, list_2)]
sums        = [a + b  for (a, b) in itertools.product(list_1, list_2)]
print("값 비교 : {}\n값의 합 : {}".format(comparisons,sums))

값 비교 : [False, False, False, False, False, False, False, False, True]
값의 합 : [3, 4, 5, 7, 8, 9, 6, 7, 8]


In [8]:
lst1 = [1,5,4]
lst2 = [2,3,4]
lst3 = list(map(lambda x: x[0] == x[1] , itertools.product(lst1,lst2)))
print('lst3 Row data : {}\nCounting "True" data : {}'.format(lst3, sum(lst3)))

lst3 Row data : [False, False, False, False, False, False, False, False, True]
Counting "True" data : 1


# **Excel to Python**
* 모듈 비교 http://www.python-excel.org/
* xlrd, openpyxl

### Django 에서의 작업
모든 작업을 한꺼번에 진행하지 말고
* 엑셀 파일을 제대로 불러왔는지를 확인하는 페이지
* 이때는 데이터 타입이 동일한 결과를 출력하는것이 연속적 3개 이상반복되면 헤더를 구분하도록 로직을 구성한다
* 이후 필요한 옵션을 설정하고 result 에서도 결과를 바꿔가며 분석 가능하도록 한다
* 결과물 또한 Excel로 다운 받을 수 있도록 한다

In [179]:
# Excel 파일을 열 때에는 xlrd 모듈만 있으면 되고
# 가공 및 저장시에는 openpyxl 모듈을 필요로 한다
import pandas as pd
import itertools
def check_xls(file, info=False):
    
    # 전체 9개 인덱스 동일내용이 있으면 1) 유효 헤더값, 2) 엑셀파일 내용 확인 모두 진행
    check_list = {"SEM" : ['가맹점번호','매입사','승인번호','원거래일자', '입금예정액', '입금예정일자','정산상태', '카드번호','할부']}    

    # 헤더와 인덱스 자동생성 없이 순수한 엑셀파일 불러오기
    data = pd.read_excel(file, header=None, index_col=None)
    
    # 유효한 컬럼값 확인
    for i_col in range(10):
        valid_data = len(data.iloc[:,i_col].dropna())
        if valid_data != 0:
            endline_col = len(data.iloc[:,i_col].dropna())
            if info: print("{} : 컬럼부터 {} 개 인덱스를 갖는 테이블".format(i_col, endline_col))
            break
            
    # 헤더값 확인하여 컬럼에 사용할 인덱스값과, 문서의 종류 확인하기
    for i in range(10):
        source_data = list(set(list(data.iloc[i,:].dropna())))
        for check in check_list.keys():
            # valid 이상 유효 값 존재여부 확인
            check_count, valid = 0, int(len(check_list[check])/2)
            check_data         = check_list[check]
            checking   = list(map(lambda x : x[0]==x[1], 
                                  itertools.product(source_data, check_data)))
            if sum(checking) > valid:
                end_i = i  # 이거없이 출력하면 9의 값을 갖는다 : break는 1번의 for만 탈출
                if info: print("이 데이터는 {}이고, 유효컬럼 라인은 {} 입니다".format(check, end_i))
                break

    # 출력본 수정1 : 컬럼수정
    col_names = data.iloc[end_i,:]
    data.columns = col_names
    # 출력본 수정2 : 유효한 값 추출 (컬럼값 + 1, 인덱스값 )
    # endline_col+1 값이 없으면 테이블 아래 다른 자료까지 포함되어 출력된다
    data = data.iloc[ end_i+1 :  endline_col+1, i_col:].reset_index()
    data = data.drop(columns = 'index')

    # 컬럼별 작업시 이름으로 호출하면 중복된 이름의 컬럼이 제대로 처리가 안된다
    # 같은 이름의 컬럼이 있으면 다르게 처리를 해보자
    col_name = data.columns
    temp   = {} # 개별 객체의 중복갯수 카운트
    rename = [] # 새로운 컬럼명
    for no, col in enumerate(col_name):
        if no == 0:
            temp[col] = 1
            rename.append(col)
        else:
            if col not in temp:
                temp[col] = 1
                rename.append(col)
            else:
                temp[col] += 1
                col = col + str(temp[col])
                rename.append(col)  
    data.columns = rename

    # 예외적인 컬럼을 제외한 나머지 숫자데이터를 , 추가하기 
    for col_name in data.columns:
        try: 
            data[col_name] = data[col_name].astype(int)
            if col_name not in ['고객ID', '발급사',  '승인번호','가맹점번호']:
                data[col_name] = data[col_name].apply("{:,}".format)
        except: pass
    return data

In [180]:
# 파일 불러오기
file = 'data/SEM_01_bak.xlsx'
df = check_xls(file, info=True).tail(2)
df

1 : 컬럼부터 3127 개 인덱스를 갖는 테이블
이 데이터는 SEM이고, 유효컬럼 라인은 2 입니다


,NO,NO2,NO3,NO4,NO5,NO6,NO7,NO8,고객ID,발급사,...,거래일자,거래시간,매입사,가맹점번호,매입일자,입금예정일자,정산상태,거래유형,수수료,입금예정액
3123,998,998,998,998,998,998,998,998,1388514922,10611052,...,2017-12-11,11:51:23,비씨,759344728,2017-12-12,2017-12-14,정산완료,승인,116,"4,884"
3124,999,999,999,999,999,999,999,999,1388514922,10611052,...,2017-12-11,11:52:38,국민,45408154,2017-12-12,2017-12-14,정산완료,승인,75,"4,925"


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 3123 to 3124
Data columns (total 31 columns):
NO        2 non-null int64
NO2       2 non-null int64
NO3       2 non-null int64
NO4       2 non-null int64
NO5       2 non-null int64
NO6       2 non-null int64
NO7       2 non-null int64
NO8       2 non-null int64
고객ID      2 non-null int64
발급사       2 non-null object
거래건수      2 non-null object
공급금액      2 non-null object
부가세       2 non-null object
거래금액      1 non-null object
카드번호      2 non-null object
봉사료       2 non-null int64
부가세2      2 non-null int64
거래금액2     2 non-null int64
승인번호      2 non-null int64
할부        2 non-null int64
원거래일자     2 non-null object
거래일자      2 non-null object
거래시간      2 non-null object
매입사       2 non-null object
가맹점번호     2 non-null int64
매입일자      2 non-null object
입금예정일자    2 non-null object
정산상태      2 non-null object
거래유형      2 non-null object
수수료       2 non-null int64
입금예정액     2 non-null int64
dtypes: int64(17), object(14)
memory usage

In [159]:
tt = ['test', 'temp', 'test']

In [160]:
tt.values

AttributeError: 'list' object has no attribute 'values'

In [157]:
rename

['NO',
 'NO1',
 'NO2',
 'NO3',
 'NO4',
 'NO5',
 'NO6',
 'NO7',
 '고객ID',
 '발급사',
 '거래건수',
 '공급금액',
 '부가세',
 '거래금액',
 '카드번호',
 '봉사료',
 '부가세16',
 '거래금액17',
 '승인번호',
 '할부',
 '원거래일자',
 '거래일자',
 '거래시간',
 '매입사',
 '가맹점번호',
 '매입일자',
 '입금예정일자',
 '정산상태',
 '거래유형',
 '수수료',
 '입금예정액']

In [76]:
for col_name in df.columns:
    try:
        df[col_name] = df[col_name].astype(int)
    except:
        pass

In [81]:
df['거래금액']

2,거래금액,거래금액
3123,NaN,5000
3124,Y,5000


In [77]:
거래금액, "봉사료", "부가세", "거래금액", "수수료", "입금예정액"

NameError: name '거래금액' is not defined

In [78]:
df.head()

2,NO,NO,NO,NO,NO,NO,NO,NO,고객ID,발급사,...,거래일자,거래시간,매입사,가맹점번호,매입일자,입금예정일자,정산상태,거래유형,수수료,입금예정액
3123,998,998,998,998,998,998,998,998,1388514922,10611052,...,2017-12-11,11:51:23,비씨,759344728,2017-12-12,2017-12-14,정산완료,승인,116,"4,884"
3124,999,999,999,999,999,999,999,999,1388514922,10611052,...,2017-12-11,11:52:38,국민,45408154,2017-12-12,2017-12-14,정산완료,승인,75,"4,925"


In [80]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>2</th>\n      <th>NO</th>\n      <th>NO</th>\n      <th>NO</th>\n      <th>NO</th>\n      <th>NO</th>\n      <th>NO</th>\n      <th>NO</th>\n      <th>NO</th>\n      <th>고객ID</th>\n      <th>발급사</th>\n      <th>거래건수</th>\n      <th>공급금액</th>\n      <th>부가세</th>\n      <th>거래금액</th>\n      <th>카드번호</th>\n      <th>봉사료</th>\n      <th>부가세</th>\n      <th>거래금액</th>\n      <th>승인번호</th>\n      <th>할부</th>\n      <th>원거래일자</th>\n      <th>거래일자</th>\n      <th>거래시간</th>\n      <th>매입사</th>\n      <th>가맹점번호</th>\n      <th>매입일자</th>\n      <th>입금예정일자</th>\n      <th>정산상태</th>\n      <th>거래유형</th>\n      <th>수수료</th>\n      <th>입금예정액</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>3123</th>\n      <td>998</td>\n      <td>998</td>\n      <td>998</td>\n      <td>998</td>\n      <td>998</td>\n      <td>998</td>\n      <td>998</td>\n      <td>998</td>\n      <td>1388514922</td>\n      <td>1

In [ ]:
df.to_html(table_id=None,
           columns=None, 
           border=None, 
           col_space=None, 
           decimal='.', 
           header=True, 
           index=True, 
           na_rep='NaN', 
           formatters=None, 
           float_format=None, 
           sparsify=None, 
           index_names=True, 
           justify=None, 
           bold_rows=True, 
           classes=None, 
           escape=True, 
           max_rows=None, 
           max_cols=None, 
           show_dimensions=False, 
           notebook=False, 
           buf=None))